In [41]:
import httpx
import json
import re

endpoint = "https://stac.eoapi.dev"
collection = "openaerialmap"

In [ ]:
resp = httpx.get(f"{endpoint}/search", params={"collections": collection})
print("Number of Items found:", resp.json()["numberMatched"])

200
Number of Items found: 16157


In [43]:
# https://github.com/kennethreitz/requests/blob/f5dacf84468ab7e0631cc61a3f1431a32e3e143c/requests/utils.py#L580-L612
def parse_header_links(value):
    """Return a dict of parsed link headers proxies.

    i.e. Link: <http:/.../front.jpeg>; rel=front; type="image/jpeg",<http://.../back.jpeg>; rel=back;type="image/jpeg"

    """

    links = []

    replace_chars = " '\""

    for val in re.split(", *<", value):
        try:
            url, params = val.split(";", 1)
        except ValueError:
            url, params = val, ""

        link = {}

        link["url"] = url.strip("<> '\"")

        for param in params.split(";"):
            try:
                key, value = param.split("=")
            except ValueError:
                break

            link[key.strip(replace_chars)] = value.strip(replace_chars)

        links.append(link)

    return links

In [57]:
%timeit

url = f"{endpoint}/search?collections={collection}&limit=1000"
while True:
    resp = httpx.get(url, headers={"Accept": "application/geo+json-seq"})
    for r in resp.iter_lines():
        _ = json.loads(r)

    links = parse_header_links(resp.headers.get("link", ""))
    url = next((link["url"] for link in links if link["rel"] == "next"), None)
    if not url:
        break

In [ ]:
%timeit

url = f"{endpoint}/search?collections={collection}&limit=1000"
while True:
    resp = httpx.get(url)
    items = resp.json()

    links = items["links"]
    url = next((link["href"] for link in links if link["rel"] == "next"), None)
    if not url:
        break

In [70]:
from geoarrow.rust.io import read_parquet
from lonboard import viz

resp = httpx.get(
    "http://127.0.0.1:8081/search?limit=1000",
    headers={"Accept": "application/vnd.apache.parquet"},
)
with open("items.parquet", "wb") as f:
    f.write(resp.content)

# Example: A GeoParquet file with Polygon or MultiPolygon geometries
table = read_parquet("items.parquet")
v = viz(table)
v

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…